<a href="https://colab.research.google.com/github/shelleyg-bit/canada-land-cover-classifier/blob/main/download_sentinel_from_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=amLHPLxf9fR2ZaBEdQNngtZUshJyt1lwkg4wVylWT-U&tc=M3b-zB3H2DYY0i0tfn38Y9-k8Ny8d5IkEMZP2kHRDIY&cc=Ez2ss-ryp3sTJ9wfL_HRLT-JVpgXlwx9CeG_k7y3afY

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiG76raa1e0ZVSjoAna4OdlpKi3tDFCNsDI09sxVf2fYyjDG3M5xVw

Successfully saved authorization token.


# Install Dependencies

In [ ]:
!pip install geopandas rasterio geemap 

In [15]:
import rasterio as rio
import geopandas as gpd
from matplotlib import pyplot as plt

In [16]:
!gdalinfo /content/drive/MyDrive/nrcan/lc_data/aoi_512x512/aoi4_lc_021 

Driver: GTiff/GeoTIFF
Files: /content/drive/MyDrive/nrcan/lc_data/aoi_512x512/aoi4_lc_021
Size is 512, 512
Coordinate System is:
PROJCS["NAD83 / Canada Atlas Lambert",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["standard_parallel_1",49],
    PARAMETER["standard_parallel_2",77],
    PARAMETER["latitude_of_origin",49],
    PARAMETER["central_meridian",-95],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","397

# Prepare BBox

In [19]:
from shapely.geometry import Polygon

def get_aoi_gdf(aoi_tifname):
  with rio.open(aoi_tifname, 'r') as src:
    l, b, r, t = src.bounds
    bbox = Polygon([(l, b), (l, t), (r, t), (r, b)])
    aoi_gdf = gpd.GeoDataFrame({'geometry': [bbox]}, crs=src.profile['crs'])
  return aoi_gdf



In [22]:
a = get_aoi_gdf('/content/drive/MyDrive/nrcan/lc_data/aoi_512x512/aoi4_lc_021')
a

,geometry
0,"POLYGON ((-119370.270 772845.289, -119370.270 ..."


# Get Sentinel Images

In [ ]:
!pip install geemap

In [25]:
# method to remove cloud from the image
def maskclouds(image: ee.Image) -> ee.Image:
    """To mask clouds using the Sentinel-2 QA band
    @param {ee.Image} image Sentinel-2 image
    @return {ee.Image} cloud masked Sentinel-2 image
    """
    band_qa = image.select('QA60')
    cloud_mask = ee.Number(2).pow(10).int()
    cirrus_mask = ee.Number(2).pow(11).int()
    mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
        band_qa.bitwiseAnd(cirrus_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def fndvi(image: ee.Image) -> ee.Image:
    """Add NDVI to each pixel in an image"""
    #TODO: ask toby about this NDVI calculation
    # since its 20m resolution
    # there are other NDVIs at 10m resolution
    ndvi = image.expression(
    "(NIR-RED)/(NIR+RED)",
    {
        'RED': image.select('B4').multiply(0.0001),
        'NIR' : image.select('B5').multiply(0.0001)
    
    });# okay
    ndf = ndvi.rename('FNDVI')
    results = ndf.copyProperties(image, ['system:time_start'])
    return image.addBands(results)


def temporal_composite(image_collection: ee.ImageCollection,
                        duration: ee.DateRange, 
                        cloud_percentage: ee.Number,
                        bands: ee.List,
                        reducer: ee.Reducer) -> ee.Image :
    ''' Applies reducer on images across a time range
    Returns a composite for the duration
    with selected bands
    '''
    ## TODO: filter out months with no data
    ## some months with many cloudy days
    ## might end up with No data
    return (image_collection.
            filterDate(duration.start(), duration.end()).
            filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)).
            map(maskclouds).
            select(bands).
            reduce(reducer))

In [26]:
def get_composites_for_aoi(time_range:ee.DateRange, aoi: ee.Geometry, out_dir: str, reproject=True, download=False) -> ee.ImageCollection:
  ''' Get sentinel images from GEE
  fitler according to bounds and duration
  and create monthly composite for all months in time_range
  Returns monthly composites collection
  Downloads collection to google drive
  '''
  collection_sentinel = ('COPERNICUS/S2_SR')
  start_date = time_range.start()
  end_date = time_range.end()
  count = end_date.difference(start_date, 'month').round().toInt()
  month_seq = ee.List.sequence(0, count.subtract(1))
  
  images_collection = ee.ImageCollection(collection_sentinel)
  
  #print(aoi.getInfo())
  # Filter collection by aoi
  images_collection_aoi = images_collection.filterBounds(aoi)

  # right projection is important for minimizing errors in computations
  projection = images_collection_aoi.first().select('B2').projection()

  def create_monthly_max_composite(month_no: ee.Number) -> ee.Image:
    ''' For a given month 
    takes the max value of each band for each pixel
    across all the images in the month
    Returns: A monthly composite with multiple bands
    '''
    month_start_date = start_date.advance(month_no, 'month')
    month_end_date = month_start_date.advance(1, 'month')
    duration = ee.DateRange(month_start_date, month_end_date)
    bands = ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])
    cloud_percent_acceptable = ee.Number(10) # % of pixels w/ clouds acceptaple in images

    image = (temporal_composite(images_collection_aoi,
                                duration, 
                                cloud_percent_acceptable,
                                bands,
                                ee.Reducer.max()).
            float().
            set('system:time_start', month_start_date.millis()).
            set('system:id', month_start_date.format('YYYYMM')).
            set('system:index', month_start_date.format('YYYYMM')))
    if (reproject):
      image = image.reproject(projection)
  
    return image
  
  # return create_monthly_max_composite(4) # for testing
  
  composite_collection = ee.ImageCollection.fromImages(month_seq.map(create_monthly_max_composite))

  if (download):
    geemap.ee_export_image_collection_to_drive(composite_collection, scale=10, descriptions=None, folder=out_dir, region=aoi)

  return composite_collection

## Download Image collection

In [39]:
import geemap
import os
from pathlib import Path

aoi_dir = '/content/drive/MyDrive/nrcan/lc_data/aoi_512x512'
image_dir = '/content/drive/MyDrive/nrcan/sentinel_aoi_images'
time_range = ee.DateRange('2020-07-01', '2020-09-01')

composite_images = []

for aoi_tifname in os.listdir(aoi_dir):
  print(aoi_tifname)
  Path(f"{image_dir}/{aoi_tifname}").mkdir(parents=True, exist_ok=True)
  aoi_gdf = get_aoi_gdf(f'{aoi_dir}/{aoi_tifname}')
  aoi_ee = geemap.geopandas_to_ee(aoi_gdf)
  aoi_geom = ee.Geometry(aoi_ee.getInfo()['features'][0]['geometry'])
  composite_images.append(get_composites_for_aoi(time_range, aoi_geom, out_dir=aoi_tifname, reproject=True, download=True))


aoi4_lc_021
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_220
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_820
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_821
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_111
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_1422
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_1425
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi4_lc_1626
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi5_lc_83
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...
aoi8_lc_48
Total number of images: 2

Exporting 202007 ...
Exporting 202008 ...


In [31]:
composite_images[9].size().getInfo()

2

In [36]:
Map = geemap.Map()
vis_params = {
    'bands': ['B8_max', 'B4_max', 'B3_max'],
    'min': 0,
    'max': 1.0
}
#vis_type = 'agriculture(B11,B8,B2)'
vis_type = 'false(B8,B4,B3)'

Map.addLayer(aoi, {'color': 'green'}, 'AOI', opacity=0.5)

image = ee.Image(composite_images[9].toList(2).get(0)) # map one image from collection
Map.addLayer(image, vis_params, vis_type) 
Map.center_object(aoi)

In [37]:

Map

Map(center=[46.35804996854766, -72.52711237054402], controls=(WidgetControl(options=['position', 'transparent_…

In [40]:
!gdalinfo /content/drive/MyDrive/nrcan/sentinel_aoi_images/aoi4_lc_021/202008.tif

Driver: GTiff/GeoTIFF
Files: /content/drive/MyDrive/nrcan/sentinel_aoi_images/aoi4_lc_021/202008.tif
Size is 1662, 1662
Coordinate System is:
PROJCS["WGS 84 / UTM zone 14N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-99],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32614"]]
Origin = (626510.000000000000000,6225720.000000000000000)
Pixel Size = (10.000000000000000

In [41]:
!gdalinfo /content/drive/MyDrive/nrcan/lc_data/aoi_512x512/aoi4_lc_021

Driver: GTiff/GeoTIFF
Files: /content/drive/MyDrive/nrcan/lc_data/aoi_512x512/aoi4_lc_021
Size is 512, 512
Coordinate System is:
PROJCS["NAD83 / Canada Atlas Lambert",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["standard_parallel_1",49],
    PARAMETER["standard_parallel_2",77],
    PARAMETER["latitude_of_origin",49],
    PARAMETER["central_meridian",-95],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","397